In [1]:
import pandas as pd
import numpy as np
import shapely.wkt
from shapely.geometry import Point, Polygon, MultiPolygon

In [2]:
file311 = "../Chicago Dataset/2019_chi_311.csv"
fileCrime = "../Chicago Dataset/data_2019.csv"
fileRegions = "../Chicago Dataset/CommAreas.csv"

In [9]:
data311 = pd.read_csv(file311)
data311.columns

Index(['Unnamed: 0', 'sr_number', 'sr_type', 'sr_short_code',
       'created_department', 'owner_department', 'status', 'origin',
       'created_date', 'last_modified_date', 'closed_date', 'street_address',
       'city', 'state', 'zip_code', 'street_number', 'street_direction',
       'street_name', 'street_type', 'duplicate', 'legacy_record',
       'community_area', 'ward', 'electricity_grid', 'police_sector',
       'police_district', 'police_beat', 'precinct', 'created_hour',
       'created_day_of_week', 'created_month', 'x_coordinate', 'y_coordinate',
       'latitude', 'longitude', 'location', ':@computed_region_rpca_8um6',
       ':@computed_region_vrxf_vc4k', ':@computed_region_6mkv_f3dw',
       ':@computed_region_bdys_3d7i', ':@computed_region_43wa_7qmu',
       'parent_sr_number'],
      dtype='object')

In [10]:
data311["sr_type"].unique()

array(['311 INFORMATION ONLY CALL', 'Street Light Out Complaint',
       'Aircraft Noise Complaint', 'Graffiti Removal Request',
       'Alley Light Out Complaint', 'Snow – Uncleared Sidewalk Complaint',
       'Traffic Signal Out Complaint', 'Pet Wellness Check Request',
       'Sanitation Code Violation', 'Open Fire Hydrant Complaint',
       'Building Violation', 'Coyote Interaction Complaint',
       'Ice and Snow Removal Request', 'Recycling Pick Up',
       'Pothole in Street Complaint', 'Stray Animal Complaint',
       'Sewer Cave-In Inspection Request',
       'Missed Garbage Pick-Up Complaint', 'Water Lead Test Kit Request',
       'Rodent Baiting/Rat Complaint', 'Garbage Cart Maintenance',
       'City Vehicle Sticker Violation', 'No Water Complaint',
       'Water Lead Test Visit Request',
       'Sewer Cleaning Inspection Request',
       'Tree Debris Clean-Up Request', 'Vicious Animal Complaint',
       'Street Light Pole Damage Complaint', 'Nuisance Animal Complaint',
   

In [11]:
data311["community_area"].unique()
comm = np.arange(1, 78)
missings = np.setdiff1d(comm, data311["community_area"].unique())
missings

array([ 9, 17, 18, 20, 32, 33, 36, 40, 41, 42, 47, 50, 54, 60, 66, 77])

In [12]:
relevantColumns311 = ['created_date','latitude','longitude','sr_type', 'community_area']
relevantComplaints311 = ['311 INFORMATION ONLY CALL', 'Aircraft Noise Complaint', 'Street Light Out Complaint', 'Alley Light Out Complaint']
# finalColumns311 = ['Created Date','Complaint Type','Precincts']

# select only the first chunk of data
data311 = pd.read_csv(file311)

# keep only the relevant columns
data311 = data311[relevantColumns311]

# keep only the relevant complaints
data311 = data311.loc[data311['sr_type'].isin(relevantComplaints311)]

print('Anomalies preprocessed!!!')
data311.shape

Anomalies preprocessed!!!


(859, 5)

In [13]:
crime_df = pd.read_csv(fileCrime)
min(crime_df["Date"])

'01/01/2019 01:00:00 AM'

In [14]:
# types_to_keep = ['THEFT', 'BATTERY', 'ASSAULT', 'CRIMINAL DAMAGE']
types_to_keep = ['THEFT', 'CRIMINAL DAMAGE', 'BATTERY', 'NARCOTICS']
crime_df = crime_df.loc[crime_df['Primary Type'].isin(types_to_keep)]

# Create Data for 1 day

In [15]:
# create a 365 x 77 x 4 matrix to store the number of crimes per day per community area per type
crime_matrix = np.zeros((365, 77, 4))

# iterate over the rows of the dataframe
for index, row in crime_df.iterrows():
    # get the date
    date = row["Date"]
    # get the community area
    community_area = int(row["Community Area"])
    # get the crime type
    crime_type = row["Primary Type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # update the matrix

    crime_matrix[day_of_year - 1, community_area - 1, types_to_keep.index(crime_type)] += 1

crime_matrix.shape

(365, 77, 4)

In [16]:
# save the matrix in a pickle file
import pickle

# with open('../Chicago Dataset/crime_matrix.pkl', 'wb') as f:
with open('../../Comparison/TEMPORAL_PRECISION/Deepcrime/1d/crime_matrix_1d.pkl', 'wb') as f:
    pickle.dump(crime_matrix, f)

FileNotFoundError: [Errno 2] No such file or directory: '../../Comparison/TEMPORAL_PRECISION/Deepcrime/1d/crime_matrix_1d.pkl'

In [15]:
# create a 365 x 77 x 4 311complaint matrix to store the number of 311 complaints per day per community area per type
complaint_matrix = np.zeros((365, 77, 4))

# iterate over the rows of the dataframe
for index, row in data311.iterrows():
    # get the date
    date = row["created_date"]
    # get the community area
    community_area = int(row["community_area"])
    # get the crime type
    complaint_type = row["sr_type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # update the matrix

    complaint_matrix[day_of_year - 1, community_area - 1, relevantComplaints311.index(complaint_type)] += 1

complaint_matrix.shape

(365, 77, 4)

In [16]:
np.count_nonzero(complaint_matrix)

50

In [17]:
# save the matrix in a pickle file
import pickle

# with open('../Chicago Dataset/complaint_matrix.pkl', 'wb') as f:
with open('../../Comparison/TEMPORAL_PRECISION/Deepcrime/1d/complaint_matrix_1d.pkl', 'wb') as f:
    pickle.dump(complaint_matrix, f)

# Create 4 Hours data

In [18]:
# create a 365 x 6 x 77 x 4 matrix to store the number of crimes per day per community area per type
# 365 x 6 denotes the 6 time slots of the day

crime_matrix = np.zeros((365, 6, 77, 4))

# iterate over the rows of the dataframe
for index, row in crime_df.iterrows():
    # get the date
    date = row["Date"]
    # get the community area
    community_area = int(row["Community Area"])
    # get the crime type
    crime_type = row["Primary Type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # get the hour of the day
    hour_of_day = pd.to_datetime(date).hour
    # update the matrix

    crime_matrix[day_of_year - 1, hour_of_day // 4, community_area - 1, types_to_keep.index(crime_type)] += 1

In [19]:
print(crime_matrix.shape)
crime_matrix = crime_matrix.reshape((-1, 77, 4))
print(crime_matrix.shape)

(365, 6, 77, 4)
(2190, 77, 4)


In [20]:
# save the matrix in a pickle file
import pickle

# with open('../Chicago Dataset/crime_matrix_4h.pkl', 'wb') as f:
with open('../../Comparison/TEMPORAL_PRECISION/Deepcrime/4h/crime_matrix_4h.pkl', 'wb') as f:
    pickle.dump(crime_matrix, f)

In [21]:
# create a 365 x 6 x 77 x 4 311complaint matrix to store the number of 311 complaints per day per community area per type
# 365 x 6 denotes the 6 time slots of the day

complaint_matrix = np.zeros((365, 6, 77, 4))

# iterate over the rows of the dataframe
for index, row in data311.iterrows():
    # get the date
    date = row["created_date"]
    # get the community area
    community_area = int(row["community_area"])
    # get the crime type
    complaint_type = row["sr_type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # get the hour of the day
    hour_of_day = pd.to_datetime(date).hour
    # update the matrix

    complaint_matrix[day_of_year - 1, hour_of_day // 4, community_area - 1, relevantComplaints311.index(complaint_type)] += 1

In [22]:
print(complaint_matrix.shape)
complaint_matrix = complaint_matrix.reshape((-1, 77, 4))
print(complaint_matrix.shape)

(365, 6, 77, 4)
(2190, 77, 4)


In [23]:
np.count_nonzero(complaint_matrix)/len(complaint_matrix)

0.028767123287671233

In [24]:
# save the matrix in a pickle file
import pickle

# with open('../Chicago Dataset/complaint_matrix_4h.pkl', 'wb') as f:
with open('../../Comparison/TEMPORAL_PRECISION/Deepcrime/4h/complaint_matrix_4h.pkl', 'wb') as f:
    pickle.dump(complaint_matrix, f)

# Create 6 Hours Data

In [25]:
# create a 365 x 4 x 77 x 4 matrix to store the number of crimes per day per community area per type
# 365 x 4 denotes the 4 time slots of the day

crime_matrix = np.zeros((365, 4, 77, 4))

# iterate over the rows of the dataframe
for index, row in crime_df.iterrows():
    # get the date
    date = row["Date"]
    # get the community area
    community_area = int(row["Community Area"])
    # get the crime type
    crime_type = row["Primary Type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # get the hour of the day
    hour_of_day = pd.to_datetime(date).hour
    # update the matrix

    crime_matrix[day_of_year - 1, hour_of_day // 6, community_area - 1, types_to_keep.index(crime_type)] += 1

In [26]:
print(crime_matrix.shape)
crime_matrix = crime_matrix.reshape((-1, 77, 4))
print(crime_matrix.shape)

(365, 4, 77, 4)
(1460, 77, 4)


In [27]:
# save the matrix in a pickle file
import pickle

# with open('../Chicago Dataset/crime_matrix_4h.pkl', 'wb') as f:
with open('../../Comparison/TEMPORAL_PRECISION/Deepcrime/6h/crime_matrix_6h.pkl', 'wb') as f:
    pickle.dump(crime_matrix, f)

In [28]:
# create a 365 x 6 x 77 x 4 311complaint matrix to store the number of 311 complaints per day per community area per type
# 365 x 6 denotes the 6 time slots of the day

complaint_matrix = np.zeros((365, 6, 77, 4))

# iterate over the rows of the dataframe
for index, row in data311.iterrows():
    # get the date
    date = row["created_date"]
    # get the community area
    community_area = int(row["community_area"])
    # get the crime type
    complaint_type = row["sr_type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # get the hour of the day
    hour_of_day = pd.to_datetime(date).hour
    # update the matrix

    complaint_matrix[day_of_year - 1, hour_of_day // 4, community_area - 1, relevantComplaints311.index(complaint_type)] += 1

In [29]:
print(complaint_matrix.shape)
complaint_matrix = complaint_matrix.reshape((-1, 77, 4))
print(complaint_matrix.shape)

(365, 6, 77, 4)
(2190, 77, 4)


In [30]:
np.count_nonzero(complaint_matrix)/len(complaint_matrix)

0.028767123287671233

In [31]:
# save the matrix in a pickle file
import pickle

# with open('../Chicago Dataset/complaint_matrix_6h.pkl', 'wb') as f:
with open('../../Comparison/TEMPORAL_PRECISION/Deepcrime/6h/complaint_matrix_6h.pkl', 'wb') as f:
    pickle.dump(complaint_matrix, f)

# Create 12 Hours data

In [39]:
# create a 365 x 12 x 77 x 4 matrix to store the number of crimes per day per community area per type
# 365 x 2 denotes the 2 time slots of the day

crime_matrix = np.zeros((365, 2, 77, 4))

# iterate over the rows of the dataframe
for index, row in crime_df.iterrows():
    # get the date
    date = row["Date"]
    # get the community area
    community_area = int(row["Community Area"])
    # get the crime type
    crime_type = row["Primary Type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # get the hour of the day
    hour_of_day = pd.to_datetime(date).hour
    # update the matrix

    crime_matrix[day_of_year - 1, hour_of_day // 12, community_area - 1, types_to_keep.index(crime_type)] += 1

In [40]:
print(crime_matrix.shape)
crime_matrix = crime_matrix.reshape((-1, 77, 4))
print(crime_matrix.shape)

(365, 2, 77, 4)
(730, 77, 4)


In [42]:
# save the matrix in a pickle file
import pickle

# with open('../Chicago Dataset/crime_matrix_12h.pkl', 'wb') as f:
with open('../../Comparison/TEMPORAL_PRECISION/Deepcrime/Dataset/12h/crime_matrix_12h.pkl', 'wb') as f:
    pickle.dump(crime_matrix, f)

In [43]:
# create a 365 x 2 x 77 x 4 311complaint matrix to store the number of 311 complaints per day per community area per type
# 365 x 2 denotes the 62time slots of the day

complaint_matrix = np.zeros((365, 2, 77, 4))

# iterate over the rows of the dataframe
for index, row in data311.iterrows():
    # get the date
    date = row["created_date"]
    # get the community area
    community_area = int(row["community_area"])
    # get the crime type
    complaint_type = row["sr_type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # get the hour of the day
    hour_of_day = pd.to_datetime(date).hour
    # update the matrix

    complaint_matrix[day_of_year - 1, hour_of_day // 12, community_area - 1, relevantComplaints311.index(complaint_type)] += 1

In [44]:
print(complaint_matrix.shape)
complaint_matrix = complaint_matrix.reshape((-1, 77, 4))
print(complaint_matrix.shape)

(365, 2, 77, 4)
(730, 77, 4)


In [45]:
np.count_nonzero(complaint_matrix)/len(complaint_matrix)

0.0684931506849315

In [46]:
# save the matrix in a pickle file
import pickle

# with open('../Chicago Dataset/complaint_matrix_12h.pkl', 'wb') as f:
with open('../../Comparison/TEMPORAL_PRECISION/Deepcrime/Dataset/12h/complaint_matrix_12h.pkl', 'wb') as f:
    pickle.dump(complaint_matrix, f)

# Create 7 Days data

In [11]:
# create a 365/7 x 77 x 4 matrix to store the number of crimes per day per community area per type
# 365/7 denotes 7 days in a week

crime_matrix = np.zeros((365//7+1, 77, 4))

# iterate over the rows of the dataframe
for index, row in crime_df.iterrows():
    # get the date
    date = row["Date"]
    # get the community area
    community_area = int(row["Community Area"])
    # get the crime type
    crime_type = row["Primary Type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # update the matrix

    crime_matrix[(day_of_year) // 7, community_area - 1, types_to_keep.index(crime_type)] += 1

In [15]:
# save the matrix in a pickle file
import pickle

with open('../Chicago Dataset/crime_matrix_7d.pkl', 'wb') as f:
    pickle.dump(crime_matrix, f)

In [13]:
# create a 365/7 x 77 x 4 311complaint matrix to store the number of 311 complaints per day per community area per type
# 365/7 denotes 7 days in a week

complaint_matrix = np.zeros((365//7+1, 77, 4))

# iterate over the rows of the dataframe
for index, row in data311.iterrows():
    # get the date
    date = row["created_date"]
    # get the community area
    community_area = int(row["community_area"])
    # get the crime type
    complaint_type = row["sr_type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # update the matrix

    complaint_matrix[(day_of_year) // 7, community_area - 1, relevantComplaints311.index(complaint_type)] += 1

In [16]:
# save the matrix in a pickle file
import pickle

with open('../Chicago Dataset/complaint_matrix_7d.pkl', 'wb') as f:
    pickle.dump(complaint_matrix, f)

# Holiday Dataset

### Upto MARCH

In [15]:
crime_df = pd.read_csv("../../Comparison/data_2020_upto_MARCH.csv")
crime_df.head()

,Date,Primary Type,Latitude,Longitude,Community Area
0,2020-03-31 23:55:00,BATTERY,41.968492,-87.703906,14.0
1,2020-03-31 23:47:00,WEAPONS VIOLATION,41.793431,-87.645136,68.0
2,2020-03-31 23:45:00,CRIMINAL DAMAGE,41.953590,-87.732989,16.0
3,2020-03-31 23:42:00,ROBBERY,41.844263,-87.723744,30.0
4,2020-03-31 23:40:00,ROBBERY,41.722134,-87.662700,73.0


In [16]:
# types_to_keep = ['THEFT', 'BATTERY', 'ASSAULT', 'CRIMINAL DAMAGE']
types_to_keep = ['THEFT', 'CRIMINAL DAMAGE', 'BATTERY', 'NARCOTICS']
crime_df = crime_df.loc[crime_df['Primary Type'].isin(types_to_keep)]
crime_df.head()

,Date,Primary Type,Latitude,Longitude,Community Area
0,2020-03-31 23:55:00,BATTERY,41.968492,-87.703906,14.0
2,2020-03-31 23:45:00,CRIMINAL DAMAGE,41.953590,-87.732989,16.0
6,2020-03-31 23:22:00,BATTERY,41.812634,-87.639447,61.0
8,2020-03-31 23:10:00,CRIMINAL DAMAGE,41.772784,-87.698107,66.0
9,2020-03-31 23:09:00,BATTERY,41.745196,-87.601375,44.0


In [17]:
# create a 365 x 77 x 4 matrix to store the number of crimes per day per community area per type
crime_matrix = np.zeros((91, 77, 4))

# iterate over the rows of the dataframe
for index, row in crime_df.iterrows():
    # get the date
    date = row["Date"]
    # get the community area
    community_area = int(row["Community Area"])
    # get the crime type
    crime_type = row["Primary Type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear
    # update the matrix

    crime_matrix[day_of_year - 1, community_area - 1, types_to_keep.index(crime_type)] += 1

crime_matrix.shape

(91, 77, 4)

In [18]:
print(crime_matrix.size)
print(np.count_nonzero(crime_matrix))

28028
13672


In [19]:
# save the matrix in a pickle file
import pickle

with open('../Chicago Dataset/test_upto_MARCH.pkl', 'wb') as f:
    pickle.dump(crime_matrix, f)

### Easter

In [20]:
crime_df = pd.read_csv("../../Comparison/easter_2020_all_crimes.csv")

# types_to_keep = ['THEFT', 'BATTERY', 'ASSAULT', 'CRIMINAL DAMAGE']
types_to_keep = ['THEFT', 'CRIMINAL DAMAGE', 'BATTERY', 'NARCOTICS']
crime_df = crime_df.loc[crime_df['Primary Type'].isin(types_to_keep)]

# create a 365 x 77 x 4 matrix to store the number of crimes per day per community area per type
crime_matrix = np.zeros((30, 77, 4))                    ### from April 15 to May 15

# iterate over the rows of the dataframe
for index, row in crime_df.iterrows():
    # get the date
    date = row["Date"]
    # get the community area
    community_area = int(row["Community Area"])
    # get the crime type
    crime_type = row["Primary Type"]
    # get the day of the year
    day_of_year = pd.to_datetime(date).dayofyear - 105          ### April 15 is 106th day of the year (2020 is a leap year)
    # update the matrix

    crime_matrix[day_of_year - 1, community_area - 1, types_to_keep.index(crime_type)] += 1

crime_matrix.shape

print(crime_matrix.size)
print(np.count_nonzero(crime_matrix))

with open('../Chicago Dataset/test_easter.pkl', 'wb') as f:
    pickle.dump(crime_matrix, f)

9240
2677


# Xmas

In [21]:
crime_df = pd.read_csv("../../Comparison/christmas_2020_all_crimes.csv")

# types_to_keep = ['THEFT', 'BATTERY', 'ASSAULT', 'CRIMINAL DAMAGE']
types_to_keep = ['THEFT', 'CRIMINAL DAMAGE', 'BATTERY', 'NARCOTICS']
crime_df = crime_df.loc[crime_df['Primary Type'].isin(types_to_keep)]

# create a 365 x 77 x 4 matrix to store the number of crimes per day per community area per type
crime_matrix = np.zeros((31, 77, 4))                    ### from Dec 15 to Jan 15

# iterate over the rows of the dataframe
for index, row in crime_df.iterrows():
    # get the date
    date = row["Date"]
    # get the community area
    community_area = int(row["Community Area"])
    # get the crime type
    crime_type = row["Primary Type"]
    # get the day of the year

    if pd.to_datetime(date).dayofyear < 350:
        day_of_year = pd.to_datetime(date).dayofyear + 15
    else:
        day_of_year = pd.to_datetime(date).dayofyear - 350

    crime_matrix[day_of_year - 1, community_area - 1, types_to_keep.index(crime_type)] += 1

crime_matrix.shape

print(crime_matrix.size)
print(np.count_nonzero(crime_matrix))

with open('../Chicago Dataset/test_xmas.pkl', 'wb') as f:
    pickle.dump(crime_matrix, f)

9548
4018
